In [10]:
import os
import numpy as np
from skimage.io import imsave, imread
import cv2

In [11]:
dim=513
data_path=r'C:\Users\nishc_omjn2ty\Downloads\SAMPLES_MULTI_CLASS'

In [12]:
train_data_path = os.path.join(data_path, 'SAMPLES-RASTER')

In [13]:
images = os.listdir(train_data_path)

In [14]:
images

['BUILTUP',
 'NON_TOF',
 'OPENLAND',
 'PLANTATION',
 'ROADS',
 'SHADOW',
 'SPARSE',
 'TOF',
 'WATER']

In [15]:
def create_train_data():
    train_data_path = os.path.join(data_path, 'SAMPLES-RASTER')
    train_data_path_mask = os.path.join(data_path, 'MASK-VALUE')
    labels = os.listdir(train_data_path)
    Num_labels = len(labels)
    imgs = []
    imgs_mask = []
    label_mask = []
    print('-'*30)
    print('Creating training images...')
    print('-'*30)
    i = 0
    for label in labels:
        images = os.listdir(os.path.join(train_data_path,label))
        masks = os.listdir(os.path.join(train_data_path_mask,label))
        total = len(images)
        res = [0]*Num_labels
        res[labels.index(label)] = 1
        res = np.array(res)
        for image_name in images:
            image_name_mask = image_name.split('.')[0] + '.png'
            img = cv2.imread(os.path.join(train_data_path,label, image_name))
            img_mask = imread(os.path.join(train_data_path_mask,label,image_name_mask),as_grey=True)
            img = np.array(img)
            img_mask = np.array(img_mask)
            img_mask=img_mask[..., np.newaxis]
            img_mask = cv2.resize(img_mask, (128,128), interpolation = cv2.INTER_AREA)
            img_mask = img_mask.astype('float32')
            img = cv2.resize(img, (128,128), interpolation = cv2.INTER_AREA)
#             img_mask = addpad_mask(img_mask)
#             img  = addpad(img)
#             result_img = np.zeros((168,168,3))
#            result_mask = np.zeros((168,168,1))
#             result_img[:img.shape[0],:img.shape[1]] = img
#            result_mask[:img_mask.shape[0],:img_mask.shape[1]] = img_mask
#             img = cv2.resize(img, (256, 256))
#             img_mask = cv2.resize(img_mask, (256, 256))
            imgs.append(img)
            imgs_mask.append(img_mask)
            label_mask.append(res)

            if i % 100 == 0:
                print('Done: {0}/{1} images'.format(i, total))
            i += 1
    print('Loading done.')
    #print(imgs[0])
    imgs= np.array(imgs)
    imgs_mask=np.array(imgs_mask)
    label_mask=np.array(label_mask)
    imgs_mask=imgs_mask[..., np.newaxis]
    return imgs,imgs_mask,label_mask


In [16]:
def load_train_data():
    return create_train_data()

In [17]:
a,b,c = load_train_data()

------------------------------
Creating training images...
------------------------------


C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\skimage\io\_io.py:48: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')


Done: 0/372 images
Done: 100/372 images
Done: 200/372 images
Done: 300/372 images
Done: 400/72 images
Done: 500/222 images
Done: 600/222 images
Done: 700/55 images
Done: 800/89 images
Done: 900/37 images
Done: 1000/476 images
Done: 1100/476 images
Done: 1200/476 images
Done: 1300/476 images
Done: 1400/40 images
Loading done.


In [59]:
def addpad(x):
    desired_size = 128
    old_size = [x.shape[0],x.shape[1]] # old_size is in (height, width) format
    if old_size[0]>desired_size:
        x = cv2.resize(x,(desired_size,old_size[1]))
        old_size[0] = desired_size
        
    if old_size[1]>desired_size:
        x = cv2.resize(x,(old_size[0],desired_size))
        old_size[1] = desired_size
        
    top = (desired_size - old_size[0])//2
    bottom = desired_size - top - old_size[0]
    left = (desired_size - old_size[1])//2
    right = desired_size - left - old_size[1]
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(x, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=color)
    return new_im

In [21]:
a = a.astype('float32')
mean = np.mean(a)  # mean for data centering
std = np.std(a)  # std for data normalization

a -= mean
a /= std

a = a.astype('float32')

In [65]:
import matplotlib.pyplot as plt
cv2.imshow('a', a[1])
#plt.show()
cv2.imshow('b', b[1])
cv2.waitKey(0)

-1

In [85]:
def addpad_mask(img):
    desired_size = 128
    old_size = list(img.shape) # old_size is in (height, width) format
    print(img.shape,old_size)
    if old_size[0]>desired_size:
        img = cv2.resize(img,(desired_size,old_size[1]))
        old_size = list(img.shape)
    print(img.shape,old_size)
    
    if old_size[1]>desired_size:
        img = cv2.resize(img,(old_size[0],desired_size))
        old_size = list(img.shape)
    print(img.shape,old_size)     
    top = (desired_size - old_size[0])//2
    bottom = desired_size - top - old_size[0]
    left = (desired_size - old_size[1])//2
    right = desired_size - left - old_size[1]
    result_mask = np.zeros((128,128,1))
    if len(img.shape)!=3:
        img = img[..., np.newaxis]
    print(old_size,img.shape)
    result_mask[top:top + img.shape[0],left:left + img.shape[1]] = img
    return result_mask

# TRAINING

In [2]:
import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, Flatten, Dense, BatchNormalization
from keras.layers.core import Dropout
from keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint
from keras import backend as K

Using TensorFlow backend.
C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\tenso

In [4]:
import tensorflow as tf

In [5]:
K.set_image_data_format('channels_last')  # TF dimension ordering in this code
img_rows = 128
img_cols = 128
smooth = 1.

In [6]:
train_datagen = ImageDataGenerator(horizontal_flip=True)
val_datagen = ImageDataGenerator()


train_image_generator = train_datagen.flow_from_directory(
r'SAMPLES_MULTI_CLASS\SAMPLES-RASTER',
    target_size=(128, 128),
batch_size = 8)

train_mask_generator = train_datagen.flow_from_directory(
r'SAMPLES_MULTI_CLASS\MASK-VALUE',
    target_size=(128, 128),
batch_size =8)

# val_image_generator = val_datagen.flow_from_directory(
# 'data/val_frames/val',
# batch_size = 8)


# val_mask_generator = val_datagen.flow_from_directory(
# 'data/val_masks/val',
# batch_size = 8)



train_generator = zip(train_image_generator, train_mask_generator)
# val_generator = zip(val_image_generator, val_mask_generator)

Found 1428 images belonging to 9 classes.
Found 1428 images belonging to 9 classes.


In [7]:
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols, imgs.shape[3]))
    for i in range(imgs.shape[0]):
        imgs_p[i] = cv2.resize(imgs[i], (img_cols, img_rows))

    #imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

In [44]:
def DiceBCELoss(targets, inputs, smooth=1e-6):    
       
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    binary_crossentropy = tf.keras.losses.BinaryCrossentropy()
    BCE =  binary_crossentropy(targets, inputs)
    intersection = K.sum(targets*inputs)   
    dice_loss = 1 - (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    Dice_BCE = BCE + dice_loss
    
    return Dice_BCE

# Model Similar to UNET

In [8]:
def get_model():
    inputs = Input((img_rows, img_cols, 3))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #84

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Dropout(0.1)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #42

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Dropout(0.1)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3) #21

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.1)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4) #7

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Dropout(0.2)(conv5)
    #pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)
    
#     conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)
#     conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv6)
    
    flat6 = Flatten()(conv5)
    dense = Dense(9, activation='softmax', name='output_1')(flat6)

#     up7 = concatenate([Conv2DTranspose(512, (2, 2), padding='same')(conv6), conv5], axis=3)
#     conv7 = Conv2D(512, (3, 3), activation='relu', padding='same')(up7)
#     conv7 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv7)
    
    up8 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Dropout(0.2)(conv8)

    up9 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv8), conv3], axis=3)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Dropout(0.1)(conv9)

    up10 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv9), conv2], axis=3)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same')(up10)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv10)
    conv10 = BatchNormalization()(conv10)
    conv10 = Dropout(0.1)(conv10)

    up11 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv10), conv1], axis=3)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same')(up11)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv11)

    conv12 = Conv2D(1, (1, 1), activation='sigmoid', name='output_2')(conv11)

    model = Model(inputs=[inputs], outputs=[conv12])

    model.compile(optimizer=Adam(lr=1e-5), loss={'output_2': 'binary_crossentropy'}, metrics=['accuracy'])

    return model


In [9]:
model = get_model()
model.summary()

W1025 22:55:47.639733 13496 deprecation_wrapper.py:119] From C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1025 22:55:47.931872 13496 deprecation_wrapper.py:119] From C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1025 22:55:47.942878 13496 deprecation_wrapper.py:119] From C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1025 22:55:48.083868 13496 deprecation_wrapper.py:119] From C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1025 22:55:48.084869 13

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 32) 9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 32) 128         conv2d_2[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [67]:
def train_and_save():
    print('-'*30)
    print('Loading and preprocessing train data...')
    print('-'*30)
    imgs_train, imgs_mask_train, imgs_label_train = load_train_data()
    
    print('Preprocessing Done...')
    imgs_train = imgs_train.astype('float32')
    #mean = np.mean(imgs_train)  # mean for data centering
    #std = np.std(imgs_train)  # std for data normalization

    #imgs_train -= mean
    #imgs_train /= std

    imgs_mask_train = imgs_mask_train.astype('float32')
    #imgs_mask_train[:,:1] /= 255.  # scale masks to [0, 1]
    
    imgs_label_train = imgs_label_train.astype('float32')

    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_model()
    #model.load_weights('weights.h5')
    model_checkpoint = ModelCheckpoint('weights_multiclass.h5', monitor='val_loss', save_best_only=True)

    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    model.fit(imgs_train, imgs_mask_train, batch_size=8, nb_epoch=200, verbose=1, shuffle=True,
              validation_split=0.2,
              callbacks=[model_checkpoint])

#     model.fit_generator(train_generator, steps_per_epoch = (332//32), epochs=400,
#               callbacks=[model_checkpoint])


In [68]:
def test_and_predict():
    print('-'*30)
    print('Loading and preprocessing test data...')
    print('-'*30)
    imgs_test = imread(os.path.join(data_path, 'RASTER/TOF_CHIPS5.jpg'))
    imgs_test = cv2.resize(imgs_test, (img_cols, img_rows))
    imgs_test = np.array([imgs_test])

    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std

    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    model = get_model()
    model.load_weights('weights_multiclass.h5')

    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    imgs_mask_test = model.predict(imgs_test, verbose=1)
    image = (imgs_mask_test[0] * 255.).astype(np.uint8)
    imsave(str(1) + '_pred.png', image)
    return image


In [69]:
train_and_save()

------------------------------
Loading and preprocessing train data...
------------------------------
------------------------------
Creating training images...
------------------------------
Done: 0/372 images
Done: 100/372 images
Done: 200/372 images
Done: 300/372 images
Done: 400/72 images
Done: 500/222 images
Done: 600/222 images
Done: 700/55 images
Done: 800/89 images
Done: 900/37 images
Done: 1000/476 images
Done: 1100/476 images
Done: 1200/476 images
Done: 1300/476 images
Done: 1400/40 images
Loading done.
Preprocessing Done...
------------------------------
Creating and compiling model...
------------------------------
------------------------------
Fitting model...
------------------------------


C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1142 samples, validate on 286 samples
Epoch 1/200
 520/1142 [============>.................] - ETA: 27s - loss: -0.1698 - acc: 0.2761ETA: 39s - loss: -0.

KeyboardInterrupt: 

In [24]:
a = test_and_predict()

------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 146ms/step


In [26]:
import cv2

imgs_test = imread(os.path.join(data_path, 'MASKS/TOF_CHIPS5.png'), as_grey=True)
imgs_test = resize(imgs_test, (img_cols, img_rows))
cv2.imshow("predicted",a[1])
cv2.imshow("ACTUAL",imgs_test)
cv2.waitKey(0)

-1

In [25]:
a=cv2.threshold(a,127,255,cv2.THRESH_BINARY)

In [21]:
a[1].shape

(256, 256)

In [152]:
imgs_test = resize(imgs_test, (1,img_cols, img_rows,1), preserve_range=True)

In [51]:
imgs_test.shape

(6719, 9872, 3)

In [50]:
cv2.imshow('Img', a)

In [6]:
import matplotlib.pyplot as plt

In [47]:
img = cv2.resize(imgs_test,(512,512))

In [79]:
cv2.imshow('img',a[0])
cv2.waitKey(0)

-1

In [41]:
x=image[1]
masks=[[]]
for i in range(1,400):
    if(i%20==0):
        if(i==20):
            masks=x
        else:
            masks= np.vstack((masks,x))
        imgs_mask_test = model.predict(np.array([a[i]]), verbose=1)
        img = (imgs_mask_test[0] * 255.).astype(np.uint8)
        img=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
        x=img[1]
        continue
    imgs_mask_test = model.predict(np.array([a[i]]), verbose=1)
    img = (imgs_mask_test[0] * 255.).astype(np.uint8)
    img=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    x = np.hstack((x,img[1]))
    
    

1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 36ms/step


In [107]:
cv2.imshow('PREDICTED_MASKS',masks)
cv2.imshow('ORIGINAL_MASKS',orig)
cv2.imshow('RGB',y)
cv2.waitKey(0)

-1

In [39]:
model = get_model()
model.load_weights('weights.h5')

imgs_mask_test = model.predict(np.array([a[0]]), verbose=1)
image = (imgs_mask_test[0] * 255.).astype(np.uint8)
image=cv2.threshold(image,127,255,cv2.THRESH_BINARY)

1/1 [==============================] - 0s 229ms/step


In [105]:
orig= cv2.resize(orig,(512,512))

In [110]:
x=a[0]
for i in range(1,400):
    if(i%20==0):
        if(i==20):
            y=x
        else:
            y= np.vstack((y,x))
        x=a[i]
        continue
    x = np.hstack((x,a[i]))

In [43]:
imsave('PREDICTED_MASKS.png',masks)

In [111]:
x=b[0]
orig=[[]]
for i in range(1,400):
    if(i%20==0):
        if(i==20):
            orig=x
        else:
            orig= np.vstack((orig,x))
        x=b[i]
        continue
    x = np.hstack((x,b[i]))

In [114]:
imsave('PREDICTED_MASKS.png',masks)
imsave('ORIGINAL_MASKS.png',orig)
imsave('RGB.png',y)

W1015 10:49:52.163126  9792 util.py:64] Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W1015 10:50:15.680769  9792 util.py:64] Lossy conversion from float32 to uint8. Range [-2.654958486557007, 1.9670199155807495]. Convert image to uint8 prior to saving to suppress this warning.


In [115]:
x=a[0]
y_1=[[]]
for i in range(1,400):
    if(i%13==0):
        if(i==13):
            y_1=x
        else:
            y_1= np.vstack((y_1,x))
        x=a[i]
        continue
    x = np.hstack((x,a[i]))

In [116]:
imsave('RGB_1.png',y_1)

W1015 11:03:53.682473  9792 util.py:64] Lossy conversion from float32 to uint8. Range [-2.654958486557007, 1.9670199155807495]. Convert image to uint8 prior to saving to suppress this warning.


In [19]:
model = get_model()
model.load_weights('weights_multiclass.h5')
from keract import get_activations, display_activations
keract_inputs = a[0]
keract_targets = b[0]
activations = get_activations(model, keract_inputs)
display_activations(activations, cmap="gray", save=False)

ValueError: tensorflow.__spec__ is None